<a href="https://colab.research.google.com/github/carlosadrianolg/meta-treinamento-ia/blob/main/analisa_notas_fiscais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ==========================================================================
# SCRIPT COMPLETO E FINAL (COM CONTAGEM CORRETA DE NOTAS)
# Copie e cole este bloco inteiro em uma célula e execute.
# ==========================================================================

# --------------------------------------------------------------------------
# PASSO 1: INSTALAÇÃO DE BIBLIOTECAS
# --------------------------------------------------------------------------
print("\n--- [1/6] Instalando bibliotecas... ---")
!pip install -q pandas==2.2.2
!pip install -q -U langchain langchain-core langchain-community langchain-google-genai streamlit pyngrok
print("✅ Bibliotecas instaladas.")

# --------------------------------------------------------------------------
# PASSO 2: CRIAÇÃO DO ARQUIVO app.py
# --------------------------------------------------------------------------
print("\n--- [2/6] Criando o arquivo app.py com lógica de contagem aprimorada... ---")

app_code = """
import streamlit as st
import pandas as pd
import os
import warnings
import io
import zipfile
import contextlib
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# --- 1. CONFIGURAÇÕES E CONSTANTES ---
st.set_page_config(page_title="Analisador de Notas Fiscais", layout="wide")
warnings.filterwarnings("ignore")

KEY_COLUMN = 'CHAVE DE ACESSO'
FORBIDDEN_KEYWORDS = [
    'import os', 'import sys', 'import subprocess', 'import shutil', 'import socket',
    'open(', 'eval(', 'exec(', '__import__', 'globals()', 'locals()',
    'requests.', 'urllib.', 'ctypes', 'pickle',
    'pd.read_csv', 'df.to_csv', 'pd.read_excel', 'to_sql'
]

# --- 2. PROMPTS DOS AGENTES ESPECIALISTAS ---

def get_code_generation_prompt(df_columns: list) -> str:
    \"\"\"Gera o prompt do sistema para o agente que cria o código Python, com lógica de contagem correta.\"\"\"

    qty_col_name = next((col for col in df_columns if 'QTD' in col.upper() or 'QUANTIDADE' in col.upper()), 'QUANTIDADE DO ITEM')
    val_col_name = next((col for col in df_columns if 'VL' in col.upper() or 'VALOR' in col.upper()), 'VALOR TOTAL DO ITEM')
    date_col_name = next((col for col in df_columns if 'DATA' in col.upper()), 'DATA DE EMISSÃO')

    # MELHORIA PRINCIPAL: Adicionado Dicionário de Análise Fiscal para diferenciar contagem de notas e de itens.
    return f'''
    Você é um analista de dados sênior, especialista em Python e Pandas para análise fiscal.
    Seu objetivo é gerar código Python para responder perguntas sobre o DataFrame `df`.

    **REGRA CRÍTICA 1: USE O `df` EXISTENTE**
    O DataFrame `df` JÁ ESTÁ CARREGADO NA MEMÓRIA. NUNCA gere código que tente ler arquivos.

    **CONTEXTO DA ESTRUTURA DOS DADOS:**
    O DataFrame `df` está no nível de **ITEM DE NOTA**. Isso significa que uma única nota fiscal (com a mesma `{KEY_COLUMN}`) pode aparecer em várias linhas se tiver vários produtos.

    **DICIONÁRIO DE ANÁLISE FISCAL (ESSENCIAL):**
    -   **Para contar NOTAS FISCAIS ÚNICAS:** Se a pergunta for sobre "quantidade de notas", "número de notas", "contagem de NF-es", use a contagem de chaves únicas: `df['{KEY_COLUMN}'].nunique()`
    -   **Para contar ITENS TOTAIS (LINHAS):** Se a pergunta for sobre "número de linhas", "total de registros" ou "itens lançados", use o tamanho do dataframe: `len(df)`
    -   **Para somar QUANTIDADES DE PRODUTOS:** Se a pergunta for sobre "unidades vendidas" ou "total de produtos", some a coluna de quantidade: `df['{qty_col_name}'].sum()`

    Você é um especialista em documentos fiscais eletrônicos (NF-e) no Brasil. Antes de executar qualquer análise, considere os seguintes significados dos campos relacionados à Nota Fiscal
    Use sempre essas definições como base para entender e interpretar os dados:
    **Dados do Cabeçalho da Nota Fiscal:
    -   **CHAVE DE ACESSO: Número único de 44 dígitos que identifica a nota fiscal eletrônica.
    -   **MODELO_cabecalho: Tipo da nota fiscal (ex: 55 = NF-e, 65 = NFC-e).
    -   **SÉRIE_cabecalho: Número de série da nota fiscal (define agrupamento lógico da numeração).
    -   **NÚMERO_cabecalho: Número sequencial da nota fiscal dentro da série.
    -   **NATUREZA DA OPERAÇÃO_cabecalho: Tipo de operação realizada (ex: Venda, Devolução, Transferência).
    -   **DATA EMISSÃO_cabecalho: Data em que a nota fiscal foi emitida.
    -   **EVENTO MAIS RECENTE: Última ocorrência registrada na nota (ex: Autorização, Cancelamento, Carta de Correção).
    -   **DATA/HORA EVENTO MAIS RECENTE: Data e hora do último evento da nota fiscal.
    -   **CPF/CNPJ Emitente_cabecalho: Número do CPF ou CNPJ de quem emitiu a nota.
    -   **RAZÃO SOCIAL EMITENTE_cabecalho: Nome empresarial do emitente.
    -   **INSCRIÇÃO ESTADUAL EMITENTE_cabecalho: Inscrição estadual do emitente (registro na SEFAZ).
    -   **UF EMITENTE_cabecalho: Estado (Unidade Federativa) do emitente.
    -   **MUNICÍPIO EMITENTE_cabecalho: Cidade do emitente.
    -   **CNPJ DESTINATÁRIO_cabecalho: CNPJ de quem está recebendo a mercadoria ou serviço.
    -   **NOME DESTINATÁRIO_cabecalho: Nome ou razão social do destinatário.
    -   **UF DESTINATÁRIO_cabecalho: Estado do destinatário.
    -   **INDICADOR IE DESTINATÁRIO_cabecalho: Informa se o destinatário é contribuinte de ICMS (ex: 1 = Contribuinte, 9 = Não Contribuinte).
    -   **DESTINO DA OPERAÇÃO_cabecalho: Indica se a operação é interna, interestadual ou para o exterior.
    -   **CONSUMIDOR FINAL_cabecalho: Indica se o destinatário é consumidor final da mercadoria (Sim/Não).
    -   **PRESENÇA DO COMPRADOR_cabecalho: Informa se a compra foi presencial, pela internet, etc.
    -   **VALOR NOTA FISCAL: Valor total da nota fiscal (soma de produtos + impostos + frete, etc).
    **Dados do Itens da Nota Fiscal:
    -   **MODELO_item: Modelo da NF ao qual o item pertence.
    -   **SÉRIE_item: Série da NF do item.
    -   **NÚMERO_item: Número da NF do item.
    -   **NATUREZA DA OPERAÇÃO_item: Tipo de operação para o item.
    -   **DATA EMISSÃO_item: Data de emissão da NF referente ao item.
    -   **CPF/CNPJ Emitente_item: CPF ou CNPJ do emitente da NF do item.
    -   **RAZÃO SOCIAL EMITENTE_item: Nome do emitente da NF do item.
    -   **INSCRIÇÃO ESTADUAL EMITENTE_item: IE do emitente da NF do item.
    -   **UF EMITENTE_item: Estado do emitente da NF do item.
    -   **MUNICÍPIO EMITENTE_item: Cidade do emitente da NF do item.
    -   **CNPJ DESTINATÁRIO_item: CNPJ do destinatário da NF do item.
    -   **NOME DESTINATÁRIO_item: Nome do destinatário da NF do item.
    -   **UF DESTINATÁRIO_item: Estado do destinatário da NF do item.
    -   **INDICADOR IE DESTINATÁRIO_item: Indica situação de ICMS do destinatário para o item.
    -   **DESTINO DA OPERAÇÃO_item: Destino da operação (Interno, Interestadual, Exterior).
    -   **CONSUMIDOR FINAL_item: Indica se o destinatário é consumidor final daquele item.
    -   **PRESENÇA DO COMPRADOR_item: Informa a forma de presença do comprador para aquele item.
    **Dados dos Produtos/Serviços:
    -   **NÚMERO PRODUTO: Número sequencial do item dentro da nota.
    -   **DESCRIÇÃO DO PRODUTO/SERVIÇO: Nome ou descrição do produto ou serviço.
    -   **CÓDIGO NCM/SH: Código fiscal NCM (Nomenclatura Comum do Mercosul) usado para classificação tributária.
    -   **NCM/SH (TIPO DE PRODUTO): Descrição ou grupo correspondente ao NCM (ex: Bebidas, Eletrônicos).
    -   **CFOP: Código Fiscal de Operações e Prestações – indica a natureza da movimentação (ex: 5102 = venda dentro do estado).
    -   **QUANTIDADE: Quantidade do item.
    -   **UNIDADE: Unidade de medida (ex: UN = unidade, KG = quilo).
    -   **VALOR UNITÁRIO: Valor de cada unidade do item.
    -   **VALOR TOTAL: Valor total do item (quantidade × valor unitário).

    **REGRA CRÍTICA 2: FORMATAÇÃO DA SAÍDA**
    Ao exibir dados com `print()`, você DEVE formatá-los para o padrão brasileiro.

    **MANUAL DE FORMATAÇÃO:**
    1.  **DATAS:** Use `.dt.strftime('%d/%m/%Y %H:%M:%S')`.
    2.  **VALORES MONETÁRIOS:** Use `.apply(lambda x: f'R$ {{x:_.2f}}'.replace('.', ',').replace('_', '.'))`.
    3.  **QUANTIDADES:** Use `.apply(lambda x: f'{{x:_.3f}}'.replace('.', ',').replace('_', '.'))`.

    **AVISO:** Aplique a formatação APENAS no resultado final. Não formate colunas que ainda serão usadas em cálculos.

    **REGRAS FINAIS:**
    1.  Sua resposta deve ser APENAS o bloco de código Python, em caso de erro, retornar a mensagem de erro.
    2.  O código DEVE usar `print()` para exibir o resultado final.
    3.  Responder somente perguntas relacionadas ao dataframe.
    '''

def get_explanation_prompt(user_prompt: str, generated_code: str, raw_answer: str) -> str:
    \"\"\"Gera o prompt para o agente de explicação.\"\"\"
    return f'''
    Você é um formatador de dados. Sua única tarefa é apresentar o resultado bruto de uma análise de dados de forma clara e profissional. Os dados já estão formatados.

    **REGRA DE OURO: NÃO ALUCINE. NÃO INVENTE DADOS. NÃO RE-FORMATE NÚMEROS OU DATAS.**

    **CONTEXTO FORNECIDO:**
    1.  **Pergunta do Usuário:** "{user_prompt}"
    2.  **Resultado Bruto do Código (Já formatado):**
        ```
        {raw_answer}
        ```

    **SUAS INSTRUÇÕES:**
    1.  Apresente o "Resultado Bruto do Código" de forma limpa e direta.
    2.  Use a "Pergunta do Usuário" para dar um título ou uma frase introdutória curta.
    3.  Use Markdown (negrito, listas) para melhorar a legibilidade.

    Agora, formate a resposta final.
    '''

# --- 3. FUNÇÕES AUXILIARES ---
@st.cache_data
def process_zip_and_load(uploaded_file):
    try:
        df_cabecalho, df_itens = None, None
        with zipfile.ZipFile(uploaded_file, 'r') as zf:
            for filename in zf.namelist():
                if not filename.endswith('.csv') or '__MACOSX' in filename: continue
                if 'cabecalho' in filename.lower() or 'header' in filename.lower():
                    with zf.open(filename) as f: df_cabecalho = pd.read_csv(f, sep=',', decimal='.')
                elif 'itens' in filename.lower() or 'items' in filename.lower():
                    with zf.open(filename) as f: df_itens = pd.read_csv(f, sep=',', decimal='.')
        if df_cabecalho is None or df_itens is None:
            st.error("ERRO: O .zip deve conter um CSV de 'cabeçalho' e um de 'itens'."); return None
        if KEY_COLUMN not in df_cabecalho.columns or KEY_COLUMN not in df_itens.columns:
            st.error(f"ERRO: A coluna de união '{KEY_COLUMN}' não foi encontrada."); return None
        df_merged = pd.merge(df_cabecalho, df_itens, on=KEY_COLUMN, how='left', suffixes=('_cabecalho', '_item'))

        for col in df_merged.columns:
            if 'DATA' in col.upper():
                try: df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')
                except Exception: pass

        st.success(f"Arquivos unidos! DataFrame com {df_merged.shape[0]} linhas e {df_merged.shape[1]} colunas.")
        return df_merged
    except Exception as e:
        st.error(f"ERRO CRÍTICO ao processar o arquivo zip: {e}"); return None

def is_code_safe(code: str) -> (bool, str):
    for keyword in FORBIDDEN_KEYWORDS:
        if keyword in code: return False, f"Execução bloqueada. Comando não permitido: `{keyword}`."
    return True, "Código seguro."

def execute_python_code(code: str, df: pd.DataFrame):
    buffer = io.StringIO()
    local_vars = {'df': df.copy(), 'pd': pd}
    try:
        with contextlib.redirect_stdout(buffer): exec(code, {}, local_vars)
        return buffer.getvalue() or "O código executou sem gerar saída."
    except Exception as e: return f"Erro ao executar o código gerado:\\n```\\n{e}\\n```"

# --- 4. LÓGICA PRINCIPAL DA APLICAÇÃO ---
def main():
    st.title("🤖 Analisador Inteligente de Notas Fiscais")
    st.write("Faça o upload de um .zip com os CSVs. A IA entende a diferença entre notas e itens.")

    if 'GOOGLE_API_KEY' not in os.environ:
        st.error("Chave de API do Google não encontrada."); st.stop()

    if "messages" not in st.session_state: st.session_state.messages = []
    if "llm" not in st.session_state: st.session_state.llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

    uploaded_zip = st.file_uploader("Anexe seu arquivo .zip aqui", type="zip")

    if uploaded_zip:
        if "df" not in st.session_state or st.session_state.get('uploaded_file_name') != uploaded_zip.name:
            with st.spinner("Processando e analisando seus arquivos..."):
                df = process_zip_and_load(uploaded_zip)
                if df is not None:
                    st.session_state.df = df
                    st.session_state.messages = [{"role": "assistant", "content": "Dados carregados! Pode perguntar."}]
                    st.session_state.uploaded_file_name = uploaded_zip.name
                    st.rerun()
                else:
                    st.session_state.pop('df', None); st.session_state.pop('uploaded_file_name', None)

    if "df" not in st.session_state:
        st.info("☝️ Aguardando o upload de um arquivo .zip para iniciar."); return

    for message in st.session_state.messages:
        with st.chat_message(message["role"]): st.markdown(message["content"])

    if prompt := st.chat_input("Quantas notas fiscais existem?"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"): st.markdown(prompt)
        with st.chat_message("assistant"): handle_chat(prompt)

def handle_chat(prompt: str):
    df = st.session_state.df
    llm = st.session_state.llm

    with st.spinner("1/3 - Gerando código Python com a lógica correta..."):
        code_prompt = get_code_generation_prompt(list(df.columns))
        llm_messages = [SystemMessage(content=code_prompt)] + [HumanMessage(content=m['content']) if m['role'] == 'user' else AIMessage(content=m['content']) for m in st.session_state.messages[-4:]]
        try:
            code_response = llm.invoke(llm_messages)
            generated_code = code_response.content.strip().replace("```python", "").replace("```", "").strip()
        except Exception as e:
            st.error(f"Erro ao gerar código: {e}"); st.stop()

    with st.spinner("2/3 - Validando e executando o código..."):
        is_safe, reason = is_code_safe(generated_code)
        if not is_safe:
            st.error(reason); st.stop()
        with st.expander("Código Gerado e Executado pela IA"): st.code(generated_code, language="python")
        raw_answer = execute_python_code(generated_code, df)

    with st.spinner("3/3 - Formatando a resposta final..."):
        explanation_prompt = get_explanation_prompt(prompt, generated_code, raw_answer)
        try:
            explanation_response = llm.invoke(explanation_prompt)
            final_explanation = explanation_response.content
        except Exception as e:
            st.error(f"Erro ao gerar a explicação final: {e}")
            final_explanation = f"**Resultado Bruto:**\\n```\\n{raw_answer}\\n```"
        st.markdown(final_explanation)
        st.session_state.messages.append({"role": "assistant", "content": final_explanation})

if __name__ == "__main__":
    main()
"""

with open('app.py', 'w', encoding='utf-8') as f:
    f.write(app_code)

print("✅ Arquivo app.py recriado com sucesso.")


# --------------------------------------------------------------------------
# PASSOS 3 a 6 (sem alterações)
# --------------------------------------------------------------------------
print("\n--- [3/6] Finalizando sessões ngrok anteriores... ---")
from pyngrok import ngrok
try:
    for tunnel in ngrok.get_tunnels(): ngrok.disconnect(tunnel.public_url)
    ngrok.kill()
except: pass
print("✅ Sessões ngrok anteriores finalizadas.")

print("\n--- [4/6] Configurando as chaves de API... ---")
from google.colab import userdata
import os
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    print("✅ Chaves de API configuradas.")
except userdata.SecretNotFoundError as e:
    print(f"❌ ERRO: Secret não encontrado: {e}."); raise

print("\n--- [5/6] Iniciando o servidor Streamlit em background... ---")
!nohup streamlit run app.py &

print("\n--- [6/6] Aguardando o servidor Streamlit iniciar (10 segundos)... ---")
import time
time.sleep(10)

print("✅ Servidor pronto. Criando o túnel público com ngrok...")
try:
    public_url = ngrok.connect(8501)
    print("\n\n======================================================================================")
    print("✅ APLICAÇÃO PRONTA!")
    print(f"Clique no link a seguir para abrir o seu aplicativo: {public_url}")
    print("======================================================================================")
except Exception as e:
    print(f"❌ ERRO AO CRIAR O TÚNEL NGROK: {e}")


--- [1/6] Instalando bibliotecas... ---


✅ Bibliotecas instaladas.

--- [2/6] Criando o arquivo app.py com lógica de contagem aprimorada... ---
✅ Arquivo app.py recriado com sucesso.

--- [3/6] Finalizando sessões ngrok anteriores... ---
✅ Sessões ngrok anteriores finalizadas.

--- [4/6] Configurando as chaves de API... ---
✅ Chaves de API configuradas.

--- [5/6] Iniciando o servidor Streamlit em background... ---
nohup: appending output to 'nohup.out'

--- [6/6] Aguardando o servidor Streamlit iniciar (10 segundos)... ---
✅ Servidor pronto. Criando o túnel público com ngrok...


✅ APLICAÇÃO PRONTA!
Clique no link a seguir para abrir o seu aplicativo: NgrokTunnel: "https://0d19-34-72-84-66.ngrok-free.app" -> "http://localhost:8501"
